<table class="ee-notebook-buttons" align="left">
    <td><a target="_blank"  href="https://github.com/giswqs/earthengine-py-notebooks/tree/master/MachineLearning/cart_classifier.ipynb"><img width=32px src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" /> View source on GitHub</a></td>
    <td><a target="_blank"  href="https://nbviewer.jupyter.org/github/giswqs/earthengine-py-notebooks/blob/master/MachineLearning/cart_classifier.ipynb"><img width=26px src="https://upload.wikimedia.org/wikipedia/commons/thumb/3/38/Jupyter_logo.svg/883px-Jupyter_logo.svg.png" />Notebook Viewer</a></td>
    <td><a target="_blank"  href="https://colab.research.google.com/github/giswqs/earthengine-py-notebooks/blob/master/MachineLearning/cart_classifier.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" /> Run in Google Colab</a></td>
</table>

## Install Earth Engine API and geemap
Install the [Earth Engine Python API](https://developers.google.com/earth-engine/python_install) and [geemap](https://github.com/giswqs/geemap). The **geemap** Python package is built upon the [ipyleaflet](https://github.com/jupyter-widgets/ipyleaflet) and [folium](https://github.com/python-visualization/folium) packages and implements several methods for interacting with Earth Engine data layers, such as `Map.addLayer()`, `Map.setCenter()`, and `Map.centerObject()`.
The following script checks if the geemap package has been installed. If not, it will install geemap, which automatically installs its [dependencies](https://github.com/giswqs/geemap#dependencies), including earthengine-api, folium, and ipyleaflet.

**Important note**: A key difference between folium and ipyleaflet is that ipyleaflet is built upon ipywidgets and allows bidirectional communication between the front-end and the backend enabling the use of the map to capture user input, while folium is meant for displaying static data only ([source](https://blog.jupyter.org/interactive-gis-in-jupyter-with-ipyleaflet-52f9657fa7a)). Note that [Google Colab](https://colab.research.google.com/) currently does not support ipyleaflet ([source](https://github.com/googlecolab/colabtools/issues/60#issuecomment-596225619)). Therefore, if you are using geemap with Google Colab, you should use [`import geemap.eefolium`](https://github.com/giswqs/geemap/blob/master/geemap/eefolium.py). If you are using geemap with [binder](https://mybinder.org/) or a local Jupyter notebook server, you can use [`import geemap`](https://github.com/giswqs/geemap/blob/master/geemap/geemap.py), which provides more functionalities for capturing user input (e.g., mouse-clicking and moving).

In [3]:
# Installs geemap package
import subprocess

try:
    import geemap
except ImportError:
    print('geemap package not installed. Installing ...')
    subprocess.check_call(["python", '-m', 'pip', 'install', 'geemap'])

# Checks whether this notebook is running on Google Colab
try:
    import google.colab
    import geemap.eefolium as geemap
except:
    import geemap

# Authenticates and initializes Earth Engine
import ee

try:
    ee.Initialize()
except Exception as e:
    ee.Authenticate()
    ee.Initialize()  
    
# Custom ISL Imports

import json
import pandas as pd
from glob import glob
import geopandas as gpd
from osgeo import gdal, osr, ogr
    

# Sample Code

## Create an interactive map 
The default basemap is `Google Maps`. [Additional basemaps](https://github.com/giswqs/geemap/blob/master/geemap/basemaps.py) can be added using the `Map.add_basemap()` function. 

In [4]:
import folium
import geemap.eefolium as gmap

Map = gmap.Map(center=[40,-100], zoom=4)
Map.setOptions('HYBRID')
# Map.setOptions('ROADMAP')
# Map.setOptions('SATELLITE')
Map.setOptions('TERRAIN')
Map.setControlVisibility(layerControl=True, fullscreenControl=True, latLngPopup=True)
Map

## Add Earth Engine Python script 

In [5]:
Map = gmap.Map(center=[40,-100], zoom=4)
# Map.setOptions('HYBRID')
# Map.setOptions('ROADMAP')
# Map.setOptions('SATELLITE')
# Map.setOptions('TERRAIN')



# Add Earth Engine dataset
# Input imagery is a cloud-free Landsat 8 composite.
l8 = ee.ImageCollection('LANDSAT/LC08/C01/T1')

image = ee.Algorithms.Landsat.simpleComposite(**{
  'collection': l8.filterDate('2018-01-01', '2018-12-31'),
  'asFloat': True
})

# Use these bands for prediction.
bands = ['B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B10', 'B11']

# Load training points. The numeric property 'class' stores known labels.
points = ee.FeatureCollection('GOOGLE/EE/DEMOS/demo_landcover_labels')

# This property of the table stores the land cover labels.
label = 'landcover'

# Overlay the points on the imagery to get training.
training = image.select(bands).sampleRegions(**{
  'collection': points,
  'properties': [label],
  'scale': 30
})

# Train a CART classifier with default parameters.
trained = ee.Classifier.cart().train(training, label, bands)

# Classify the image with the same bands used for training.
classified = image.select(bands).classify(trained)

# Display the inputs and the results.
Map.centerObject(points, 11)
Map.addLayer(image, {'bands': ['B4', 'B3', 'B2'], 'max': 0.4}, 'image')
Map.addLayer(classified,
             {'min': 0, 'max': 2, 'palette': ['red', 'green', 'blue']},
             'classification')



In [6]:
l8.size().getInfo()

1136307

In [7]:
d = training.getInfo()

In [8]:
set([i["properties"]["landcover"] for i in d["features"]])

{0, 1, 2}

In [9]:
points.getInfo()

{'type': 'FeatureCollection',
 'columns': {'landcover': 'Long', 'system:index': 'String'},
 'version': 1569893036369951,
 'id': 'GOOGLE/EE/DEMOS/demo_landcover_labels',
 'properties': {'system:asset_size': 14523},
 'features': [{'type': 'Feature',
   'geometry': {'type': 'Point',
    'coordinates': [-122.27096557617189, 37.820452055421086]},
   'id': '00000000000000000003',
   'properties': {'landcover': 0}},
  {'type': 'Feature',
   'geometry': {'type': 'Point',
    'coordinates': [-122.2723388671875, 37.828587698949825]},
   'id': '00000000000000000004',
   'properties': {'landcover': 0}},
  {'type': 'Feature',
   'geometry': {'type': 'Point',
    'coordinates': [-122.28195190429686, 37.82696064199382]},
   'id': '00000000000000000005',
   'properties': {'landcover': 0}},
  {'type': 'Feature',
   'geometry': {'type': 'Point',
    'coordinates': [-122.28126525878908, 37.81882481909333]},
   'id': '00000000000000000006',
   'properties': {'landcover': 0}},
  {'type': 'Feature',
   'geo

In [10]:
d["features"][0]["properties"]

{'B10': 302.1449890136719,
 'B11': 300.9042663574219,
 'B2': 0.13984587788581848,
 'B3': 0.11473792791366577,
 'B4': 0.1099824607372284,
 'B5': 0.11954239010810852,
 'B6': 0.12579503655433655,
 'B7': 0.1057196781039238,
 'landcover': 0}

In [11]:
trained.confusionMatrix().array().getInfo()

[[32, 0, 0], [0, 32, 0], [0, 0, 34]]

In [12]:
classified.getInfo()

{'type': 'Image',
 'bands': [{'id': 'classification',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': -2147483648,
    'max': 2147483647},
   'crs': 'EPSG:4326',
   'crs_transform': [1, 0, 0, 0, 1, 0]}]}

## Display Earth Engine data layers 

In [13]:
help(Map.setControlVisibility)

Help on method set_control_visibility in module geemap.eefolium:

set_control_visibility(layerControl=True, fullscreenControl=True, latLngPopup=True) method of geemap.eefolium.Map instance
    Sets the visibility of the controls on the map.
    
    Args:
        layerControl (bool, optional): Whether to show the control that allows the user to toggle layers on/off. Defaults to True.
        fullscreenControl (bool, optional): Whether to show the control that allows the user to make the map full-screen. Defaults to True.
        latLngPopup (bool, optional): Whether to show the control that pops up the Lat/lon when the user clicks on the map. Defaults to True.



In [14]:
# Map.addLayerControl() # This line is not needed for ipyleaflet-based Map.
Map.setControlVisibility(layerControl=True, fullscreenControl=True, latLngPopup=True)
Map

# ISL Classification

In [15]:
def output_intersecting_chips(raster_path,vector_path,output_path_yes=None,output_path_no=None,copy_chips=True):
    
    raster_list = glob(raster_path + "/*.tif")
    
    vector = ogr.Open(vector_path)
    
    geometries = {}

    geometries["yes"] = {"polygons": {"objs": [],"coords": [] } ,"points":{"objs": [],"coords":[] }}
    
    geometries["no"] = {"polygons": {"objs": [],"coords": [] } ,"points":{"objs": [],"coords":[] }}
    
    features = []

    # Get vector geometry
    layer = vector.GetLayer()
    feature_count = vector.GetLayer().GetFeatureCount()

    for feature_number in range(feature_count):
        feature = layer.GetFeature(feature_number)
        vectorGeometry = feature.GetGeometryRef()

        for raster_path in raster_list:
            raster = gdal.Open(raster_path)
    

            # Get raster geometry
            transform = raster.GetGeoTransform()
            pixelWidth = transform[1]
            pixelHeight = transform[5]
            cols = raster.RasterXSize
            rows = raster.RasterYSize

            xLeft = transform[0]
            yTop = transform[3]
            xRight = xLeft+cols*pixelWidth
            yBottom = yTop+rows*pixelHeight

            ring = ogr.Geometry(ogr.wkbLinearRing)
            ring.AddPoint(xLeft, yTop)
            ring.AddPoint(xLeft, yBottom)
            ring.AddPoint(xRight, yBottom)
            ring.AddPoint(xRight, yTop)
            ring.AddPoint(xLeft, yTop)
            rasterGeometry = ogr.Geometry(ogr.wkbPolygon)
            rasterGeometry.AddGeometry(ring)

            # If intersection detected, copy chip to destination directory
            if rasterGeometry.Intersect(vectorGeometry):
                if copy_chips:
                    out_file = output_path_yes + raster_path.split("/")[-1]
                    copyfile(raster_path,out_file)
                raster_list.remove(raster_path)
                geometries["yes"]["polygons"]["objs"].append(rasterGeometry.ExportToJson())
                geometries["yes"]["points"]["objs"].append(rasterGeometry.Centroid().ExportToJson())
    
    
    for no_raster_path in raster_list:
        
        raster = gdal.Open(no_raster_path)
        # Get raster geometry
        transform = raster.GetGeoTransform()
        pixelWidth = transform[1]
        pixelHeight = transform[5]
        cols = raster.RasterXSize
        rows = raster.RasterYSize

        xLeft = transform[0]
        yTop = transform[3]
        xRight = xLeft+cols*pixelWidth
        yBottom = yTop+rows*pixelHeight

        ring = ogr.Geometry(ogr.wkbLinearRing)
        ring.AddPoint(xLeft, yTop)
        ring.AddPoint(xLeft, yBottom)
        ring.AddPoint(xRight, yBottom)
        ring.AddPoint(xRight, yTop)
        ring.AddPoint(xLeft, yTop)
        rasterGeometry = ogr.Geometry(ogr.wkbPolygon)
        rasterGeometry.AddGeometry(ring)
        
#         polygons_no.append(rasterGeometry.ExportToJson())
        geometries["no"]["polygons"]["objs"].append(rasterGeometry.ExportToJson())
        geometries["no"]["points"]["objs"].append(rasterGeometry.Centroid().ExportToJson())
        
        
#         if copy_chips:
#             out_file = output_path_no + no_raster_path.split("/")[-1]
#             copyfile(raster_path,out_file)

    objs_lists = { 
    1 : geometries["yes"]["polygons"]["objs"],
    2 : geometries["yes"]["points"]["objs"],  
    3 : geometries["no"]["polygons"]["objs"],
    4 : geometries["no"]["points"]["objs"]
            }
    
    coords_lists = { 
    1 : geometries["yes"]["polygons"]["coords"],
    2 : geometries["yes"]["points"]["coords"],  
    3 : geometries["no"]["polygons"]["coords"],
    4 : geometries["no"]["points"]["coords"]
            }
    
    
    for k in objs_lists:
        objs_lists[k] = list(set(objs_lists[k]))
        
    for k in coords_lists:
        if k == 1 or k == 3: 
            coords = [json.loads(i)["coordinates"][0][0][0:2] for i in objs_lists[k]] 
            coords_lists[k].append(coords)
        else:
            coords = [json.loads(i)["coordinates"] for i in geometries["yes"]["points"]["objs"]]
            coords_lists[k].append(coords)
            

    
#     coords_yes = [json.loads(i)["coordinates"][0][0][0:2] for i in polygons_yes] 
#     coords_no = [json.loads(i)["coordinates"][0][0][0:2] for i in polygons_no] 
    
#     polygons["yes"] = coords_yes
#     polygons["no"] = coords_no
    
# #     print(polygons["yes"])
    
    
#     for k,v in polygons.items():
#         for polygon in v:            
#             polygon = ee.Geometry.Polygon(v)
#             if k == "yes":
#                 feature = ee.Feature(polygon,{"class":1})
#             if k == "no":
#                 feature = ee.Feature(polygon,{"class":0})
#             features.append(feature)


    for k,v in geometries.items():
        for key in geometries[k]:
            if key == "points":
                for point in geometries[k][key]["coords"][0]:
                    point = ee.Geometry.Point(point)
                    if k == "yes":
                        feature = ee.Feature(point,{"class":1})
#                         print(feature.getInfo())
                    if k == "no":
                        feature = ee.Feature(point,{"class":0})
                    features.append(feature)
                    
                    
        
#         for polygon in v:            
#             polygon = ee.Geometry.Polygon(v)
#             if k == "yes":
#                 feature = ee.Feature(polygon,{"class":1})
#             if k == "no":
#                 feature = ee.Feature(polygon,{"class":0})
#             features.append(feature)


    fc = ee.FeatureCollection(features)
    
#     return geometries
    
    
    return fc
                
                
                
raster_path = "/Volumes/Lacie/zhenyadata/Project_Canopy_Data/PC_Data/Sentinel_Data/GEE/tiles_v3/polygon_exports/v8_dynamic_date_range_v4_native_secondary_sort_using_area/chips/full/2/"
vector_path = "/Volumes/Lacie/zhenyadata/Project_Canopy_Data/PC_Data/Geometry/test_line_labelling/v2/ISL_2-ML/ISL_Roads_2.shp"
output_path_yes = "/Volumes/Lacie/zhenyadata/Project_Canopy_Data/PC_Data/Sentinel_Data/GEE/tiles_v3/polygon_exports/v8_dynamic_date_range_v4_native_secondary_sort_using_area/chips/yes/2/"
output_path_no = "/Volumes/Lacie/zhenyadata/Project_Canopy_Data/PC_Data/Sentinel_Data/GEE/tiles_v3/polygon_exports/v8_dynamic_date_range_v4_native_secondary_sort_using_area/chips/no/2/"

In [16]:
fc = output_intersecting_chips(raster_path,vector_path,output_path_yes,output_path_no,copy_chips=False)

# Couldn't use linestrings due to too many points for google earth engine processing

In [410]:
# Couldn't use linestrings due to too many points for google earth engine processing

# # for line_obj in gdf["geometry"]:
# #     for line in line_obj.coords.xy:
# #         print(line.tolist())

# lines = []
# for index,row in gdf.iterrows():
#     line_coords = []
#     line_obj = gdf.loc[index]["geometry"]
#     line_obj_coords = line_obj.coords.xy
#     for index in range(len(line_obj_coords[0])):
#         x = line_obj_coords[0][index]
#         y = line_obj_coords[1][index]
#         line_coords.append([x,y])
#     lines.append(line_coords)
    
# features = []
# for line in lines:
#     gee_lines = ee.Geometry.LineString(line)
#     feature = ee.Feature(gee_lines,{"land_classification":1})
#     features.append(feature)
    
# fc = ee.FeatureCollection(features)
        

In [411]:
# Couldn't use because too many points for google earth engine processing

# def fc_from_shp(shp_dir):

#     shp_list = glob(shape_dir + "/*.shp")
    
#     gdf = pd.concat([gpd.read_file(shp) for shp in shp_list]).pipe(gpd.GeoDataFrame).reset_index()
    
#     lines = []
#     for index,row in gdf.iterrows():
#         line_coords = []
#         line_obj = gdf.loc[index]["geometry"]
#         line_obj_coords = line_obj.coords.xy
#         for index in range(len(line_obj_coords[0])):
#             x = line_obj_coords[0][index]
#             y = line_obj_coords[1][index]
#             line_coords.append([x,y])
#         lines.append(line_coords)

#     features = []
#     for line in lines:
#         gee_lines = ee.Geometry.LineString(line)
#         feature = ee.Feature(gee_lines,{"class":1})
#         features.append(feature)

#     fc = ee.FeatureCollection(features)
    
#     return [lines,fc,gdf]
    

In [412]:
# shp_dir = "/Users/purgatorid/Downloads/shapefiles/ISL_2-ML"

# fc = fc_from_shp(shp_dir)[1]

In [413]:
# gdf = fc_from_shp(shp_dir)[2]

In [414]:
# gdf.to_file("/Volumes/Lacie/zhenyadata/Project_Canopy_Data/PC_Data/Geometry/test_line_labelling/v2/ISL_2-ML/ISL_Roads_2.shp",driver='ESRI Shapefile')

In [415]:
# lines = fc_from_shp(shp_dir)[0]

In [416]:
# lens = []
# for line in lines:
#     lens.append(len(line))

In [417]:
# sum(lens) / len(lens)

In [418]:
# len(lens)

In [419]:
# col = ee.ImageCollection('users/zwarshavsky/rasters_ISL')

In [420]:
# img = ee.Image('users/zwarshavsky/rasters_ISL/2_ISL')

In [421]:
# help(ee.Classifier.cart().train)

In [422]:
# s2 = ee.ImageCollection('users/zwarshavsky/2_roads_ISL_rasters')

In [423]:
# for footprint in s2.aggregate_array('system:footprint').getInfo():
# #     print(footprint["coordinates"])
#     poly = ee.Geometry.Polygon(footprint["coordinates"])
#     feature = ee.Feature(gee_lines,{"class":1})
#     features.append(feature)

# fc = ee.FeatureCollection(features)
    

In [424]:
# for image_id in range(s2.size().getInfo()): 
#     s2.first().getInfo()["properties"]['system:footprint']["coordinates"]

In [425]:
# img = s2.mosaic()

In [17]:
ee.Image('users/zwarshavsky/rasters_ISL/2_ISL').getInfo()

{'type': 'Image',
 'bands': [{'id': 'b1',
   'data_type': {'type': 'PixelType', 'precision': 'float'},
   'dimensions': [1153, 1104],
   'crs': 'EPSG:4326',
   'crs_transform': [8.983152841195215e-05,
    0,
    8.87158208290757,
    0,
    -8.983152841195215e-05,
    5.804643871395112]},
  {'id': 'b2',
   'data_type': {'type': 'PixelType', 'precision': 'float'},
   'dimensions': [1153, 1104],
   'crs': 'EPSG:4326',
   'crs_transform': [8.983152841195215e-05,
    0,
    8.87158208290757,
    0,
    -8.983152841195215e-05,
    5.804643871395112]},
  {'id': 'b3',
   'data_type': {'type': 'PixelType', 'precision': 'float'},
   'dimensions': [1153, 1104],
   'crs': 'EPSG:4326',
   'crs_transform': [8.983152841195215e-05,
    0,
    8.87158208290757,
    0,
    -8.983152841195215e-05,
    5.804643871395112]},
  {'id': 'b4',
   'data_type': {'type': 'PixelType', 'precision': 'float'},
   'dimensions': [1153, 1104],
   'crs': 'EPSG:4326',
   'crs_transform': [8.983152841195215e-05,
    0,
   

In [18]:
Map = gmap.Map(center=[40,-100], zoom=4)

# Use these bands for prediction.
bands = ['b1','b2','b3','b4','b5','b9']

# Custom Image from my collection
img = ee.Image('users/zwarshavsky/rasters_ISL/2_ISL')

# Load training points. The numeric property 'class' stores known labels.
fc = fc

# This property of the table stores the land cover labels.
label = 'class'

# # Overlay the points on the imagery to get training.
training = img.sampleRegions(**{
  'collection': fc,
  'properties': [label],
  'scale': 10
})

# Train a CART classifier with default parameters.
trained = ee.Classifier.svm().train(training, label, bands)

# Classify the image with the same bands used for training.
classified = img.select(bands).classify(trained)

# Display the inputs and the results.
Map.centerObject(fc, 11)
Map.addLayer(img, {'bands': ['b6', 'b7', 'b8'],"max": 3000, "gamma": [4, 4, 4]})
# Map.addLayer(training, {'bands': ['b6', 'b7', 'b8'], 'max': 0.4}, 'image')
Map.addLayer(classified,
             {'min': 0, 'max': 1, 'palette': ['red', 'green']},
             'classification')


In [19]:
Map.setControlVisibility(layerControl=True, fullscreenControl=True, latLngPopup=True)
Map

In [20]:
classified.max().getInfo()

EEException: Required argument (image2) missing to function: Selects the maximum of the first and second values for each matched pair of
bands in image1 and image2. If either image1 or image2 has only 1 band,
then it is used against all the bands in the other image. If the images
have the same number of bands, but not the same names, they're used
pairwise in the natural order. The output bands are named for the longer of
the two inputs, or if they're equal in length, in image1's order. The type
of the output pixels is the union of the input types.

Args:
  image1: The image from which the left operand bands are taken.
  image2: The image from which the right operand bands are taken.

In [21]:
classified.toArray().getInfo()

{'type': 'Image',
 'bands': [{'id': 'array',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': -2147483648,
    'max': 2147483647,
    'dimensions': 1},
   'dimensions': [1153, 1104],
   'crs': 'EPSG:4326',
   'crs_transform': [8.983152841195215e-05,
    0,
    8.87158208290757,
    0,
    -8.983152841195215e-05,
    5.804643871395112]}],
 'properties': {'system:footprint': {'type': 'LinearRing',
   'coordinates': [[8.87161571959976, 5.705425487320564],
    [8.871626997140938, 5.705424756964786],
    [8.975112875478366, 5.705424778713734],
    [8.97515459009085, 5.705461384720051],
    [8.975200518326108, 5.705492443485174],
    [8.975202625126533, 5.7055035322911225],
    [8.9752033835747, 5.705514801047476],
    [8.9752033835747, 5.804598978619143],
    [8.975166640983018, 5.80464038434263],
    [8.97513537608478, 5.804686159238848],
    [8.97512045995172, 5.804688942675791],
    [8.871626997140938, 5.804688925076663],
    [8.87158533757529, 5.804652408632864],

In [22]:
help(ee.Classifier)

Help on class Classifier in module ee:

class Classifier(ee.computedobject.ComputedObject)
 |  Classifier(*args, **kwargs)
 |  
 |  A representation of an Earth Engine computed object.
 |  
 |  This is a base class for most API objects.
 |  
 |  The class itself is not abstract as it is used to wrap the return values of
 |  algorithms that produce unrecognized types with the minimal functionality
 |  necessary to interact well with the rest of the API.
 |  
 |  ComputedObjects come in two flavors:
 |  1. If func != null and args != null, the ComputedObject is encoded as an
 |     invocation of func with args.
 |  2. If func == null and args == null, the ComputedObject is a variable
 |     reference. The variable name is stored in its varName member. Note that
 |     in this case, varName may still be null; this allows the name to be
 |     deterministically generated at a later time. This is used to generate
 |     deterministic variable names for mapped functions, ensuring that nested

In [437]:
trained.confusionMatrix().getInfo()

[[0, 50], [0, 50]]

In [439]:
training.getInfo()

{'type': 'FeatureCollection',
 'columns': {},
 'properties': {'band_order': ['b1',
   'b2',
   'b3',
   'b4',
   'b5',
   'b6',
   'b7',
   'b8',
   'b9']},
 'features': [{'type': 'Feature',
   'geometry': None,
   'id': '1_0',
   'properties': {'b1': 177,
    'b2': 333,
    'b3': 293,
    'b4': 1632,
    'b5': 1759,
    'b6': 30,
    'b7': 34,
    'b8': 19,
    'b9': 0.6955844163894653,
    'class': 1}},
  {'type': 'Feature',
   'geometry': None,
   'id': '2_0',
   'properties': {'b1': 201,
    'b2': 396,
    'b3': 291,
    'b4': 2453,
    'b5': 2471,
    'b6': 30,
    'b7': 41,
    'b8': 20,
    'b9': 0.7879008650779724,
    'class': 1}},
  {'type': 'Feature',
   'geometry': None,
   'id': '3_0',
   'properties': {'b1': 159,
    'b2': 309,
    'b3': 227,
    'b4': 1696,
    'b5': 2137,
    'b6': 24,
    'b7': 32,
    'b8': 16,
    'b9': 0.7639105319976807,
    'class': 1}},
  {'type': 'Feature',
   'geometry': None,
   'id': '4_0',
   'properties': {'b1': 183,
    'b2': 339,
    'b3'

# Visualize Image Collection

In [40]:
basemap_col = ee.ImageCollection("users/zwarshavsky/basemap_v1")


In [41]:
basemap_mosaic = basemap_col.mosaic()

In [45]:
visualize_raster(basemap_mosaic)

# Sandbox

In [44]:
import folium
import geemap.eefolium as gmap

TCI_RGB = ['b1', 'b2', 'b3']
vis = {'bands': TCI_RGB,"max": 3000, "gamma": [4, 4, 4]}


def visualize_geo(coords):
    Map = gmap.Map()
    geo_obj = ee.Geometry.Polygon(coords)
    Map.centerObject(geo_obj,3)
    Map.add_layer(geo_obj, {}, 'default display')
    Map.add_child(folium.LayerControl())
    return Map

def visualize_raster(img):
    Map = gmap.Map()
    Map.centerObject(img,10)
    Map.addLayer(img, vis)
    Map.add_child(folium.LayerControl())
    return Map

def visualize_col(col):
    Map = gmap.Map()
    mosaic = col.mosaic()
    Map.centerObject(mosaic,10)
    Map.addLayer(mosaic, vis)
    Map.add_child(folium.LayerControl())
    return Map

In [25]:
col = ee.ImageCollection("users/zwarshavsky/rasters_ISL")
col.get(3).getInfo()

EEException: Element.get, argument 'property': Invalid type.
Expected type: String.
Actual type: Integer.
Actual value: 3

In [508]:
help(col)

Help on ImageCollection in module ee.imagecollection object:

class ImageCollection(ee.collection.Collection)
 |  ImageCollection(*args, **kwargs)
 |  
 |  Representation for an Earth Engine ImageCollection.
 |  
 |  Method resolution order:
 |      ImageCollection
 |      ee.collection.Collection
 |      ee.element.Element
 |      ee.computedobject.ComputedObject
 |      ee.encodable.Encodable
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  And = reduce.and(*args, **kwargs)
 |      Reduces an image collection by setting each pixel to 1 iff all the non-
 |      masked values at that pixel are non-zero across the stack of all matching
 |      bands. Bands are matched by name.
 |      
 |      Args:
 |        collection: The image collection to reduce.
 |  
 |  Or = reduce.or(*args, **kwargs)
 |      Reduces an image collection by setting each pixel to 1 iff any of the non-
 |      masked values at that pixel are non-zero across the stack of all matching
 |      bands. Ba

In [26]:
img = ee.ImageCollection("users/zwarshavsky/rasters_ISL").first()


# ee.Image("users/zwarshavsky/rasters_ISL/2_ISL").getInfo()

# img = ee.Image("users/zwarshavsky/rasters_ISL/2_ISL")

visualize_raster(img)

In [27]:
help(Map.addLayer)

Help on method add_layer in module geemap.eefolium:

add_layer(ee_object, vis_params={}, name='Layer untitled', shown=True, opacity=1.0, **kwargs) method of geemap.eefolium.Map instance
    Adds a given EE object to the map as a layer.
    
    Args:
        ee_object (Collection|Feature|Image|MapId): The object to add to the map.
        vis_params (dict, optional): The visualization parameters. Defaults to {}.
        name (str, optional): The name of the layer. Defaults to 'Layer untitled'.
        shown (bool, optional): A flag indicating whether the layer should be on by default. Defaults to True.
        opacity (float, optional): The layer's opacity represented as a number between 0 and 1. Defaults to 1.

